In [1]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup as bs
import lxml
import numpy as np
import unicodedata
import pandas as pd
from pymongo import MongoClient

In [7]:
client = MongoClient('localhost', 27017)
db = client['vacancy_db']
hh = db.head_hunter
sj = db.superjob
def vacancy_db():
    search_result_sj = pd.DataFrame(columns = db.superjob.find()[0].keys())
    search_result_hh = pd.DataFrame(columns = db.head_hunter.find()[0].keys())
    search_resolve_sj = db.superjob.find()
    search_resolve_hh = db.head_hunter.find()
    i=0
    while i!=-1:
        try:
            for n_col,column in enumerate(search_result_sj.columns):
                search_result_sj.loc[i,column] = list(search_resolve_sj[i].values())[n_col]
            i+=1
        except Exception:
            i=-1            
    i=0
    while i!=-1:
        try:
            for n_col,column in enumerate(search_result_hh.columns):
                search_result_hh.loc[i,column] = list(search_resolve_hh[i].values())[n_col]
            i+=1
        except Exception:
            i=-1
    return search_result_hh,search_result_sj

vacancy_hh,vacancy_sj = vacancy_db()
vacancy_df = vacancy_sj.append(vacancy_hh).reset_index().drop('index',axis=1)
vacancy_df.drop('_id',axis=1,inplace=True)
vacancy_df.head(50)

,vacancy_name,salary_min,salary_max,salary_currency,link,source_domain,IDs
0,Аналитик баз данных,35000,NaN,руб.,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33436564
1,Аналитик данных Junior,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33543720
2,Аналитик данных,40000,40000,руб.,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33566753
3,Аналитик баз данных (SQL),NaN,80000,руб.,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,32325125
4,Аналитик данных (информатик),NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33591345
5,"Аналитик мастер-данных (MDM, НСИ)",NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33523965
6,Менеджер-аналитик по сбору данных,40000,50000,руб.,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33561590
7,Руководитель отдела аналитики,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33597671
8,Аналитик,NaN,40500,руб.,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,32141740
9,Ведущий аналитик Центра внедрения и сопровожде...,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,31654461


In [22]:
vacancy_df['salary_min'] = vacancy_df['salary_min'].astype(float)
vacancy_df['salary_max'] = vacancy_df['salary_max'].astype(float)
vacancy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2202 entries, 0 to 2201
Data columns (total 7 columns):
vacancy_name       2202 non-null object
salary_min         580 non-null float64
salary_max         377 non-null float64
salary_currency    2202 non-null object
link               2202 non-null object
source_domain      2202 non-null object
IDs                2202 non-null object
dtypes: float64(2), object(5)
memory usage: 120.5+ KB


In [11]:
main_link_HH = 'https://hh.ru/search/vacancy'
main_link_SJ = 'https://russia.superjob.ru/vacancy/search'#?keywords
header = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
# получение пользовательского запроса
# обработка запроса - приведение к виду, который используется на сайте
search_string = input()
search_query_HH = '+'.join(search_string.split())
search_query_SJ = '%20'.join(search_string.split())

#####
# получение результатов запроса с сайта hh.ru
# bs_result - результат разбора первого запроса
# все обработанные bs() результаты для первого источника сохраняем в список bs_result
# на сайте hh.ru при нахождении определенного блока на странице можно сделать вывод, что вакансии на этой странице есть;
# поэтому сохраним такую страницу для последующего разбора
# если получена ссылка в строке с get(href) - значит, информация о вакансии на странице есть - и мы получаем еще одну разобранную функцией bs страницу
# сохраняем информацию в виде списка для разбора вакансий hh.ru
#####
request_result_HH = requests.get(main_link_HH+'?text='+search_query_HH, headers=header)
bs_result_HH = bs(request_result_HH.text,'lxml')
bs_result_list_HH = []
bs_result_list_HH.append(bs_result_HH.find('div',{'class':'vacancy-serp'}))
i=1
while i:
  try:
    bs_result_HH = bs(requests.get(main_link_HH+'?text='+search_query_HH+'&page='+str(i), headers=header).text,'lxml')
    bs_result_vacancies_HH = bs_result_HH.find('div',{'class':'vacancy-serp'})
    bs_result_vacancies_HH.find('a',{'data-qa':'vacancy-serp__vacancy-title'}).get('href')
    bs_result_list_HH.append(bs_result_vacancies_HH)
    i+=1
  except Exception:
    break

#####
# получение результатов запроса с сайта superjob.ru
# получение результатов со второго источника тем же методом - и сохранение их в список bs_result2
# номер последней страницы можно узнать из paging внизу страницы
# 0 и 1 страницы на superjob - это одно и то же; поэтому, после получения первых результатов (первая страница) переходим ко второй (2)
# в получившемся page_list_SJ 9 "лишних" элементов; если элементов больше, значит, есть и другие, кроме проверенной, страницы
#####   
request_result_SJ = requests.get(main_link_SJ+'?keywords='+search_query_SJ, headers=header)
bs_result_SJ = bs(request_result_SJ.text,'lxml')
bs_result_list_SJ = []
bs_result_list_SJ.append(bs_result_SJ)
page_list_SJ = bs_result_SJ.find_all('span',{'class':'_3IDf-'})
i=2
if any('>1</span>' in str(i) for i in page_list_SJ):
  last_page = int(str(page_list_SJ[-3:-1][0]).split('>')[1].split('<')[0])
  while i <= last_page:
    bs_result_SJ = bs(requests.get(main_link_SJ+'?keywords='+search_query_SJ+'&page='+str(i), headers=header).text,'lxml')
    bs_result_list_SJ.append(bs_result_SJ)
    i+=1

 аналитик данных


In [15]:
# создаем списки для сохранения данных о зарплате
# минимальная зарплата - min_salary
# максимальная зарплата - max_salary
# информация о заработной плате сохраняется в salary_data (со всех источников, по порядку)
# валюта, в которой платится зарплата - currency
# список вакансий: vac_list_title_HH для HeadHunter, vac_list_title_SJ для SuperJob
# список всех вакансий: vac_list_title (со всех источников, по порядку)
# итоговый датафрейм: result
##### Зарплата: разбор
# для каждого элемента в получающемся списке проверям 4 условия
# 1 - если строка с информацией о зарплате начинается со слова "до": создаем список данных разбиением по пробелу;
# "до" удаляем; последний элемент списка = валюта; все, что до него - зарплата
# 2 - та же логика при наличии слова 'от'.
# 3 - если в тексте строки есть знак '-', значит, указана зарплата "от и до":
    # первый элемент - это минимальная зарплата;
    # второй - максимальная;
    #последний элемент после сплита максимальной зарплаты по пробелу - это название валюты
# 4 - последний вариант: если ничего не указано, значит, данных нет:
    # вставляем значение np.nan для данных о зарплате, пустая строка - для списка валют.

####
# HeadHunter
####
min_salary = []
max_salary = []
currency = []
vac_list_title_HH=[]
vac_list_title_SJ=[]
id_hh = list(vacancy_df.loc[vacancy_df['source_domain']=='HeadHunter','IDs'])
id_sj = list(vacancy_df.loc[vacancy_df['source_domain']=='SuperJob','IDs'])
for bs_result_vacancies_HH in bs_result_list_HH:
    for i in bs_result_vacancies_HH.find_all('a',{'data-qa':'vacancy-serp__vacancy-title'}):
        link = i.get('href').split('?query')[0]
        vacancy_id_hh = link.split('vacancy/')[-1].split('&place')[0]
        id_hh.append(link.split('vacancy/')[-1].split('&place')[0])
        if vacancy_id_hh not in id_hh:
            vac_list_title_HH.append([link,i.getText(),'HeadHunter',link.split('vacancy/')[-1].split('&place')[0]])
    for i in bs_result_vacancies_HH.find_all('div',{'class':'vacancy-serp-item__row vacancy-serp-item__row_header'}):
        if vacancy_id_hh not in id_hh:
            vac_salary = i.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'})
            if vac_salary:
                salary = unicodedata.normalize('NFKD',vac_salary.getText())
                if salary[0:2]=='до':
                    max_salary.append(''.join(salary.split(' ')[:-1]).replace('до',''))
                    currency.append(salary.split(' ')[-1])
                    min_salary.append(np.nan)
                elif salary[0:2]=='от':
                    min_salary.append(''.join(salary.split(' ')[:-1]).replace('от',''))
                    currency.append(salary.split(' ')[-1])
                    max_salary.append(np.nan)
                elif '-' in salary:
                    min_salary.append(''.join(salary.split('-')[0].split(' ')))
                    currency.append(salary.split('-')[-1].split(' ')[-1])
                    max_salary.append(''.join(salary.split('-')[1].split(' ')[:-1]))
            else:
                min_salary.append(np.nan)
                max_salary.append(np.nan)
                currency.append('')
####
# SuperJob
####
for bs_result_SJ in bs_result_list_SJ:
    # по какой-то причине теги на superjob меняются; поэтому обрабатываются сразу 2 варианта, которые были выявлены при перехвате ошибок
    try:
        bs_result_vacancies_SJ=bs_result_SJ.find_all('div',{'class':'f-test-vacancy-item'})
    except Exception:
        pass
    if len(bs_result_vacancies_SJ)==0:
        try:
            bs_result_vacancies_SJ=bs_result_SJ.find_all('div',{'class':'_3syPg _3P0J7 _9_FPy'})
        except Exception:
            pass
    for i in bs_result_vacancies_SJ:
        try:
            link = main_link_SJ+i.find('a',{'class':'icMQ_'}).get('href')
            if link.split('.html')[0].split('-')[-1] not in vac_list_title_SJ:
                vac_list_title_SJ.append([link,i.find('div',{'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).getText(),'SuperJob',link.split('.html')[0].split('-')[-1]])
        except Exception:
            link = main_link_SJ+i.find('a',{'class':'icMQ_'}).get('href')
            if link.split('.html')[0].split('-')[-1] not in vac_list_title_SJ:
                vac_list_title_SJ.append([link,i.find('span',{'class':'_1rS-s'}).getText(),'SuperJob',link.split('.html')[0].split('-')[-1]])
    salaries_SJ=bs_result_SJ.find_all('span',{'class':'f-test-text-company-item-salary'})
    for vac_salary_SJ in salaries_SJ:
        if link.split('.html')[0].split('-')[-1] not in vac_list_title_SJ:
            vac_salary_SJ = unicodedata.normalize("NFKD",vac_salary_SJ.getText())
            if vac_salary_SJ[0:3]=='до ':
                max_salary.append(''.join(vac_salary_SJ.split(' ')[:-1]).replace('до',''))
                currency.append(vac_salary_SJ.split(' ')[-1])
                min_salary.append(np.nan)
            elif vac_salary_SJ[0:3]=='от ':
                min_salary.append(''.join(vac_salary_SJ.split(' ')[:-1]).replace('от',''))
                currency.append(vac_salary_SJ.split(' ')[-1])
                max_salary.append(np.nan)
            elif '—' in vac_salary_SJ:
                min_salary.append(''.join(vac_salary_SJ.split('—')[0].split(' ')))
                currency.append(vac_salary_SJ.split('—')[-1].split(' ')[-1])
                max_salary.append(''.join(vac_salary_SJ.split('—')[1].split(' ')[:-1]))
            elif vac_salary_SJ[:10] == 'По договор':
                min_salary.append(np.nan)
                max_salary.append(np.nan)
                currency.append('')
            elif len(vac_salary_SJ)>1:
                min_salary.append(''.join(vac_salary_SJ.split(' ')[:-1]))
                max_salary.append(''.join(vac_salary_SJ.split(' ')[:-1]))
                currency.append(vac_salary_SJ.split(' ')[-1])
            else:
                min_salary.append(vac_salary_SJ)
                max_salary.append(vac_salary_SJ)
                currency.append('')

salary_data=pd.DataFrame([min_salary,max_salary,currency]).T.rename(columns = {0:'min_salary',1:'max_salary',2:'currency'})
result_list = []
vac_list_title_HH.extend(vac_list_title_SJ)
vac_list_title = vac_list_title_HH
i=len(vac_list_title)
for k in range(i):
    result_list.append([vac_list_title[k][1],\
                        salary_data['min_salary'][k],\
                        salary_data['max_salary'][k],\
                        salary_data['currency'][k],\
                        vac_list_title[k][0],\
                        vac_list_title[k][2],\
                        vac_list_title[k][3],\
                        search_string])
result_for_new_vacancies = pd.DataFrame(result_list).rename(columns={0:'Вакансия',1:'Мин.зарплата',2:'Макс.зарплата',3:'Валюта',4:'Ссылка',5:'Сервис',6:'ID',7:'Поисковый запрос'})

In [16]:
result_for_new_vacancies

,Вакансия,Мин.зарплата,Макс.зарплата,Валюта,Ссылка,Сервис,ID,Поисковый запрос
0,Аналитик данных Junior,NaN,NaN,,https://russia.superjob.ru/vacancy/search/clie...,SuperJob,297921,аналитик данных
1,Аналитик баз данных,35000,NaN,₽,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33436564,аналитик данных
2,Аналитик данных,40000,40000,₽,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33566753,аналитик данных
3,Аналитик баз данных (SQL),NaN,80000,₽,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,32325125,аналитик данных
4,Аналитик данных (информатик),NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33591345,аналитик данных
...,...,...,...,...,...,...,...,...
196,Маркетолог,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33596774,аналитик данных
197,Маркетолог в сеть магазинов Велосалон,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33596892,аналитик данных
198,Специалист по маркетингу и рекламе,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33342624,аналитик данных
199,Менеджер по работе с сервисами регистрации,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob,33449039,аналитик данных


In [18]:
result_HH = result_for_new_vacancies.loc[result_for_new_vacancies['Сервис']=='HeadHunter',:].reset_index().drop('index',axis=1)
result_SJ = result_for_new_vacancies.loc[result_for_new_vacancies['Сервис']=='SuperJob',:].reset_index().drop('index',axis=1)

i=0
while i < len(result_HH):
    hh.insert_one({'vacancy_name':result_HH['Вакансия'][i],
                   'salary_min':result_HH['Мин.зарплата'][i],
                   'salary_max':result_HH['Макс.зарплата'][i],
                   'salary_currency':result_HH['Валюта'][i],
                   'link':result_HH['Ссылка'][i],
                   'source_domain':result_HH['Сервис'][i],
                   'IDs':result_HH['ID'][i],
                   'Search_query':result_HH['Поисковый запрос'][i]})
    i+=1

i=0
while i < len(result_SJ):
    sj.insert_one({'vacancy_name':result_SJ['Вакансия'][i],
                   'salary_min':result_SJ['Мин.зарплата'][i],
                   'salary_max':result_SJ['Макс.зарплата'][i],
                   'salary_currency':result_SJ['Валюта'][i],
                   'link':result_SJ['Ссылка'][i],
                   'source_domain':result_SJ['Сервис'][i],
                   'IDs':result_SJ['ID'][i],
                   'Search_query':result_SJ['Поисковый запрос'][i]})
    i+=1

In [23]:
vacancy_hh,vacancy_sj = vacancy_db()
vacancy_df = vacancy_sj.append(vacancy_hh).reset_index().drop('index',axis=1)
vacancy_df.drop('_id',axis=1,inplace=True)
vacancy_df['salary_min'] = vacancy_df['salary_min'].astype(float)
vacancy_df['salary_max'] = vacancy_df['salary_max'].astype(float)
vacancy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2403 entries, 0 to 2402
Data columns (total 8 columns):
vacancy_name       2403 non-null object
salary_min         653 non-null float64
salary_max         421 non-null float64
salary_currency    2403 non-null object
link               2403 non-null object
source_domain      2403 non-null object
IDs                2403 non-null object
Search_query       2403 non-null object
dtypes: float64(2), object(6)
memory usage: 150.3+ KB
